In [1]:
import ray
import json

from byzer_llm import chat
from byzerllm.utils.basic_llm_utils import (
    ChatMessage,
    MessageRole
)
from evaluate import (
    QuestionCategory,
    load_question_data,
    save_df_to_excel,
    questions_to_dataframe
)

ray.init("ray://172.16.10.245:10001", namespace="default", ignore_reinit_error=True)

2023-11-14 16:27:42,175	INFO client_builder.py:243 -- Passing the following kwargs to ray.init() on the server: ignore_reinit_error


Python version:,3.10.11
Ray version:,2.7.0
Dashboard:,http://172.16.10.245:8265


In [6]:
QUESTION_GEN_V1 = """I want you to act as a curious questioner and come up with a new question based \
 on the original problem and context. 
 
Step by step, what additional information do you need to know to better understand the \
 situation? What other perspectives or angles could be explored? How might this issue \
 impact different groups of people? Use your creativity and critical thinking skills to \
 craft a thought-provoking question that will inspire further discussion and exploration.
  
Finally, generate at least 10 new questions, each with no more than 15 words. Generate in Chinese. \
 You must strictly use the following JSON format to return:
 ```
 [
    {
 
        "new_question": "here is new question",
        "new_question_answer": "generate answers based on new questions and contexts"
    }
    ...
 ]
 ```

Original Question: %s
Context: %s 
New Questions: 
"""


# all_questions = load_question_data("data/question_v20.xlsx", drop_duplicates=True)
# 
# intention_questions = []
# for q in all_questions:
#     if q.category == QuestionCategory.INTENTION.value:
#         intention_questions.append(q)
# 
# save_df_to_excel(questions_to_dataframe(intention_questions))

In [8]:
intentions_questions_temp = load_question_data("data/output_1699929713.xlsx")

INPUT_PREFIX = """Give suggestion of options provided. 
Based on user input:"""

for i, q in enumerate(intentions_questions_temp):
    messages = q.messages
    final_gen_prompt = None
    # gen new question, prepare gen prompt
    try:
        if messages and messages[-1].content:
            old_q_content = messages[-1].content[len(INPUT_PREFIX):].strip()
            prompt = QUESTION_GEN_V1 % (
                old_q_content,
                messages[0].content
            )
            q.gen_prompt = prompt
    except Exception as e:
        print(f"failed to generate new question for the [{i}] question: {e}")
        continue

if len(intentions_questions_temp) > 0:
    q_df = questions_to_dataframe(intentions_questions_temp)
    save_df_to_excel(q_df, "data/intention_question_v3_temp.xlsx")

load [2605] questions data from data/output_1699929713.xlsx
save [2605] data to [data/intention_question_v3_temp.xlsx] done!


In [3]:
import copy

intention_questions = load_question_data("intention_gen_questions_v1.xlsx")
new_intentions_questions = []
bad_questions = []

INPUT_PREFIX = """Give suggestion of options provided. 
Based on user input:"""

for i, q in enumerate(intention_questions):
    messages = q.messages
    final_gen_prompt = None
    try:
        if messages and messages[-1].content:
            old_q_content = messages[-1].content[len(INPUT_PREFIX):].strip()
            new_questions = json.loads(q.gen_questions)
            for new_q_content in new_questions:
                new_q = copy.deepcopy(q)
                new_q.messages.pop()
                new_q.messages.append(ChatMessage(
                    role=MessageRole.USER,
                    content=INPUT_PREFIX + " " + new_q_content["new_question"]
                ))
                new_q.set_answer(new_q_content["new_question_answer"])
                new_intentions_questions.append(new_q)
    except Exception as e:
        print(f"failed to generate new question for the [{i}] question: {e}")
        bad_questions.append(q)
        continue

if len(new_intentions_questions) > 0:
    q_df = questions_to_dataframe(new_intentions_questions)
    save_df_to_excel(q_df, "data/intention_question_v3.xlsx")

if len(bad_questions) > 0:
    q_df = questions_to_dataframe(bad_questions)
    save_df_to_excel(q_df, "data/intention_question_v3_bad.xlsx")

load [2605] questions data from intention_gen_questions_v1.xlsx
failed to generate new question for the [0] question: Extra data: line 1 column 2 (char 1)
{'new_question': '这些省份和城市的销售额在过去十年中有何变化？', 'new_question_answer': '[JUST_CHAT]'}
{'new_question': '这些省份和城市的销售额与其他因素的关系如何？', 'new_question_answer': '[METRIC_RECOMMENDATION]'}
{'new_question': '这些省份和城市的销售额是否受到政策变化的影响？', 'new_question_answer': '[JUST_CHAT]'}
{'new_question': '这些省份和城市的销售额是否与人口密度有关？', 'new_question_answer': '[METRIC_RECOMMENDATION]'}
{'new_question': '这些省份和城市的销售额是否与经济发展水平有关？', 'new_question_answer': '[METRIC_RECOMMENDATION]'}
{'new_question': '这些省份和城市的销售额是否受到季节性影响？', 'new_question_answer': '[METRIC_RECOMMENDATION]'}
{'new_question': '这些省份和城市的销售额是否受到市场竞争的影响？', 'new_question_answer': '[METRIC_RECOMMENDATION]'}
{'new_question': '这些省份和城市的销售额是否与消费者偏好有关？', 'new_question_answer': '[METRIC_RECOMMENDATION]'}
{'new_question': '这些省份和城市的销售额是否受到商品价格的影响？', 'new_question_answer': '[METRIC_RECOMMENDATION]'}
{'new_question': '这些省份和城市的销售额是

In [ ]:
chat("你好", model="azure_openai")